In [10]:
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
from scipy.stats import sem, t
from skopt import gp_minimize
import optuna
import random
import itertools
from math import log

In [27]:
def get_city_coord_dict(filename):
    '''Opens the file with cities and corresponding coordinates. Puts it in an ordered dictionary.
    Input: file name with path.
    Ouput: Ordered dictionary of cities with corresponding x and y coordinates in a numpy array.'''
    city_coord_dict = {}

    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if line[0].isdigit():
                split = line.split()

                city_coord_dict[int(split[0])] = np.array([float(coord) for coord in line.split()[-2:]])

    return OrderedDict(city_coord_dict)

#Open initial file, store cities with coordinates
filename = "TSP-Configurations/eil51.tsp.txt"
filename = "TSP-Configurations/a280.tsp.txt"
filename = "TSP-Configurations/pcb442.tsp.txt"

init_cities = get_city_coord_dict(filename)

def get_distance(city_a, city_b):
    '''
    Input: numpy arrays or tuples of city_a and city_b coordinates.
    Output: Euclidean distance between the two cities (as a scalar)
    '''
    city_a, city_b = np.array(city_a), np.array(city_b)
    return np.linalg.norm(city_a - city_b)

def total_distance(cities):
    '''
    Input: OrderedDict with city names as keys and coordinates as values.
    Output: Total distance of the path that visits all cities once and returns to the starting city.
    '''
    city_coords = np.array(list(cities.values()))

    # Init variable: link last city with the first
    total_distance = get_distance(city_coords[-1], city_coords[0])

    # Connect every subsequent pair of cities
    for i in range(len(city_coords) - 1):
        total_distance += get_distance(city_coords[i], city_coords[i + 1])

    return total_distance


def visualize_routes(cities):
    '''Visualize the cities on a plane and the routes between cities.
    Input: OrderedDict of cities (keys) and coordinates(values).
    Output: plot of cities and routes between them.
    '''
    city_coords = list(cities.values())
    for city in city_coords:
        plt.scatter(city[0], city[1])

    #Add connecting lines 
    for i in range(len(city_coords) - 1):
        plt.plot([city_coords[i][0], city_coords[i+1][0]], [city_coords[i][1], city_coords[i+1][1]])

    #Connect last city to first
    plt.plot([city_coords[-1][0], city_coords[0][0]], [city_coords[-1][1], city_coords[0][1]])
    plt.show()

# Nearest Neighbour 

In [28]:
class SimulatedAnnealing:
    def __init__(self, cities, C, T0, max_step=100, init_seed=1056):
        self.cities_old = cities
        self.C = C
        self.T0 = T0
        self.max_step = max_step
        self.init_seed = init_seed
        self.step = 0

    def nearest_neighbor_heuristic(self):
        cities = list(self.cities_old.keys())
        current_city = random.choice(cities)
        solution = [current_city]

        while len(solution) < len(cities):
            distances = [(city, np.linalg.norm(np.array(self.cities_old[current_city]) - np.array(self.cities_old[city])))
                         for city in cities if city not in solution]
            next_city, _ = min(distances, key=lambda x: x[1])
            solution.append(next_city)
            current_city = next_city

        return OrderedDict((city, self.cities_old[city]) for city in solution)

    def proposal(self):
        # Select a random key from the dictionary
        random_key = np.random.choice(list(self.cities_old.keys()))
        value = self.cities_old[random_key]

        proposal_cities = self.cities_old.copy()
        del proposal_cities[random_key]

        #Move it to here     
        new_index = np.random.randint(0, len(self.cities_old) - 1)

        # Create a new dictionary with the rearranged order
        proposal_cities = OrderedDict(list(self.cities_old.items())[:new_index] + [(random_key, value)] + list(self.cities_old.items())[new_index:])
        return proposal_cities


    def evaluate(self, cities, proposal_cities, T):
        delta_distance = total_distance(proposal_cities) - total_distance(cities)
        alpha_func = min(np.exp(-delta_distance / T), 1)
        return alpha_func

    def select(self, alpha_func, proposal_cities):
        u = np.random.uniform()
        if u <= alpha_func:
            cities_new = proposal_cities
        else:
            cities_new = self.cities_old
        return cities_new

    def run(self):
        # Initial heuristic solution (nearest neighbor)
        initial_solution = self.nearest_neighbor_heuristic()
        self.cities_old = initial_solution

        while self.step < self.max_step:
            self.init_seed += 1
            T = (self.C * np.log(self.step + self.T0))**(-1)
            self.step += 1
            proposal_cities = self.proposal()
            alpha_func = self.evaluate(self.cities_old, proposal_cities, T)
            cities_new = self.select(alpha_func, proposal_cities)
            self.cities_old = cities_new

        return cities_new


In [19]:
#Run the simulator
if __name__== '__main__':
    sim_annealing = SimulatedAnnealing(cities = init_cities, C = 0.8077887004677304, T0 = 50, max_step = 100)
    cities_new = sim_annealing.run()
    visualize_routes(cities_new)
    print(f'Total distance of simulated annealing solution is: {total_distance(cities_new)}')
    print(f'Order of the cities is as follows: {cities_new.keys()}')

In [34]:
# Load the TSP instances
eil51 = get_city_coord_dict("TSP-Configurations/eil51.tsp.txt")
a280 = get_city_coord_dict("TSP-Configurations/a280.tsp.txt")
pcb442 = get_city_coord_dict("TSP-Configurations/pcb442.tsp.txt")

# Define a function to run hyperparameter optimization for a given TSP instance
def optimize_tsp_instance(instance, study_name):
    # Define the objective function
    def objective(trial):
        # Define the hyperparameter search space
        T0 = trial.suggest_float('T0', 1, 10000)
        C = trial.suggest_float('C', 0.01, 1.0)
        max_step = trial.suggest_int('max_step', 10, 1000)

        # Create and run the SimulatedAnnealing instance with the chosen hyperparameters
        sa = SimulatedAnnealing(instance, C, T0, max_step)
        optimized_cities = sa.run()

        # Calculate the objective value (in this case, the total distance)
        min_distance = total_distance(optimized_cities)

        return min_distance

    # Create an Optuna study
    study = optuna.create_study(direction='minimize', study_name=study_name)

    # Optimize the hyperparameters
    study.optimize(objective, n_trials=50)  # You can increase the number of trials for better results

    # Print the best hyperparameters and objective value
    print(f"Best trial for {study_name}:")
    best_trial = study.best_trial
    print("  Value: {:.2f}".format(best_trial.value))
    print("  Params: ")
    for key, value in best_trial.params.items():
        print("    {}: {}".format(key, value))

# Run hyperparameter optimization for each TSP instance
optimize_tsp_instance(eil51, "eil51_tsp_optimization")
optimize_tsp_instance(a280, "a280_tsp_optimization")
optimize_tsp_instance(pcb442, "pcb442_tsp_optimization")

[I 2023-12-14 21:31:52,821] A new study created in memory with name: eil51_tsp_optimization
[I 2023-12-14 21:31:52,937] Trial 0 finished with value: 630.6418802819694 and parameters: {'T0': 545.0653576085851, 'C': 0.7804602807162574, 'max_step': 155}. Best is trial 0 with value: 630.6418802819694.
[I 2023-12-14 21:31:53,527] Trial 1 finished with value: 516.7606278536034 and parameters: {'T0': 5227.119457365936, 'C': 0.43110217031499826, 'max_step': 915}. Best is trial 1 with value: 516.7606278536034.
[I 2023-12-14 21:31:53,781] Trial 2 finished with value: 544.9729726958225 and parameters: {'T0': 2676.1383549749644, 'C': 0.12503023270763086, 'max_step': 343}. Best is trial 1 with value: 516.7606278536034.
[I 2023-12-14 21:31:53,986] Trial 3 finished with value: 720.7794646388469 and parameters: {'T0': 1247.9290183573855, 'C': 0.013253120040138837, 'max_step': 274}. Best is trial 1 with value: 516.7606278536034.
[I 2023-12-14 21:31:54,416] Trial 4 finished with value: 511.7215343524971

[I 2023-12-14 21:32:07,604] Trial 39 finished with value: 540.1834889874249 and parameters: {'T0': 4982.597416220258, 'C': 0.41757640429851584, 'max_step': 530}. Best is trial 6 with value: 496.3843478637582.
[I 2023-12-14 21:32:07,841] Trial 40 finished with value: 525.4853611273811 and parameters: {'T0': 6479.195474111684, 'C': 0.12787620414805914, 'max_step': 336}. Best is trial 6 with value: 496.3843478637582.
[I 2023-12-14 21:32:08,357] Trial 41 finished with value: 532.073553315055 and parameters: {'T0': 4493.67755018424, 'C': 0.9487858095652519, 'max_step': 737}. Best is trial 6 with value: 496.3843478637582.
[I 2023-12-14 21:32:08,650] Trial 42 finished with value: 525.3594812450763 and parameters: {'T0': 3926.740370256377, 'C': 0.9985233447293246, 'max_step': 433}. Best is trial 6 with value: 496.3843478637582.
[I 2023-12-14 21:32:09,260] Trial 43 finished with value: 519.7089961083499 and parameters: {'T0': 5438.3729877576525, 'C': 0.8457909984125983, 'max_step': 854}. Best i

Best trial for eil51_tsp_optimization:
  Value: 490.95
  Params: 
    T0: 7894.682162905283
    C: 0.8974853576167918
    max_step: 493


[I 2023-12-14 21:32:13,438] Trial 0 finished with value: 3488.0351793848663 and parameters: {'T0': 7710.452260043245, 'C': 0.23993239418493595, 'max_step': 343}. Best is trial 0 with value: 3488.0351793848663.
[I 2023-12-14 21:32:16,593] Trial 1 finished with value: 3181.586506884611 and parameters: {'T0': 5345.997694248345, 'C': 0.8504269787754118, 'max_step': 797}. Best is trial 1 with value: 3181.586506884611.
[I 2023-12-14 21:32:19,199] Trial 2 finished with value: 3331.1073848356004 and parameters: {'T0': 8340.78802329789, 'C': 0.529278188647089, 'max_step': 696}. Best is trial 1 with value: 3181.586506884611.
[I 2023-12-14 21:32:20,777] Trial 3 finished with value: 3436.804159608838 and parameters: {'T0': 1493.2196881017667, 'C': 0.20163771148359513, 'max_step': 397}. Best is trial 1 with value: 3181.586506884611.
[I 2023-12-14 21:32:21,602] Trial 4 finished with value: 3205.778964197487 and parameters: {'T0': 6207.747587551339, 'C': 0.14241636789473988, 'max_step': 165}. Best is

[I 2023-12-14 21:34:15,248] Trial 39 finished with value: 3273.3310797877343 and parameters: {'T0': 2436.2495206342132, 'C': 0.8848764337737579, 'max_step': 19}. Best is trial 19 with value: 3049.4570702619276.
[I 2023-12-14 21:34:17,862] Trial 40 finished with value: 3307.964337418607 and parameters: {'T0': 4419.747749790088, 'C': 0.9564323736915232, 'max_step': 226}. Best is trial 19 with value: 3049.4570702619276.
[I 2023-12-14 21:34:21,502] Trial 41 finished with value: 3326.8680658389594 and parameters: {'T0': 3235.2872882822444, 'C': 0.9046552826961015, 'max_step': 367}. Best is trial 19 with value: 3049.4570702619276.
[I 2023-12-14 21:34:24,453] Trial 42 finished with value: 3596.791245006306 and parameters: {'T0': 2628.8086388346046, 'C': 0.9444697854144015, 'max_step': 317}. Best is trial 19 with value: 3049.4570702619276.
[I 2023-12-14 21:34:28,377] Trial 43 finished with value: 3527.1215551646173 and parameters: {'T0': 3833.279959051437, 'C': 0.8803065756006356, 'max_step': 

Best trial for a280_tsp_optimization:
  Value: 3049.46
  Params: 
    T0: 4371.8793675683855
    C: 0.9855031390846619
    max_step: 469


[I 2023-12-14 21:34:55,533] Trial 0 finished with value: 62161.05304201338 and parameters: {'T0': 2225.3842464444115, 'C': 0.24859757311954112, 'max_step': 587}. Best is trial 0 with value: 62161.05304201338.
[I 2023-12-14 21:35:01,004] Trial 1 finished with value: 62182.00580927058 and parameters: {'T0': 8825.673042805533, 'C': 0.15628002817521405, 'max_step': 759}. Best is trial 0 with value: 62161.05304201338.
[I 2023-12-14 21:35:05,797] Trial 2 finished with value: 63860.73441270661 and parameters: {'T0': 9384.460043068108, 'C': 0.3116267251236504, 'max_step': 573}. Best is trial 0 with value: 62161.05304201338.
[I 2023-12-14 21:35:08,655] Trial 3 finished with value: 63404.60005991797 and parameters: {'T0': 730.8909508024057, 'C': 0.1155411346238107, 'max_step': 324}. Best is trial 0 with value: 62161.05304201338.
[I 2023-12-14 21:35:14,548] Trial 4 finished with value: 60837.04953500077 and parameters: {'T0': 198.86949216657692, 'C': 0.5918763796444297, 'max_step': 815}. Best is 

[I 2023-12-14 21:37:54,334] Trial 40 finished with value: 61571.446386871816 and parameters: {'T0': 6538.539828433996, 'C': 0.18327057491252427, 'max_step': 636}. Best is trial 21 with value: 59141.24487713975.
[I 2023-12-14 21:37:58,485] Trial 41 finished with value: 65137.181040228956 and parameters: {'T0': 4390.46677453226, 'C': 0.02997913355029017, 'max_step': 528}. Best is trial 21 with value: 59141.24487713975.
[I 2023-12-14 21:38:02,021] Trial 42 finished with value: 62451.32565842702 and parameters: {'T0': 4545.233678567463, 'C': 0.10515069632845121, 'max_step': 392}. Best is trial 21 with value: 59141.24487713975.
[I 2023-12-14 21:38:07,089] Trial 43 finished with value: 62363.78284619073 and parameters: {'T0': 3707.6362034010626, 'C': 0.051579586851408635, 'max_step': 720}. Best is trial 21 with value: 59141.24487713975.
[I 2023-12-14 21:38:10,881] Trial 44 finished with value: 61429.18336077727 and parameters: {'T0': 5296.432541410732, 'C': 0.13398972383094584, 'max_step': 4

Best trial for pcb442_tsp_optimization:
  Value: 59141.24
  Params: 
    T0: 3915.2633066647077
    C: 0.017156029726929506
    max_step: 482
